In [7]:
# import modules
from datasets import load_dataset
from transformers import LlamaForCausalLM, LlamaTokenizer
import argparse
import socket
from dataset import get_dataset, build_dataset, tokenize
import torch
import os
import json
import pickle

In [3]:
path = './dataset'
if not os.path.exists(path):
    os.mkdir(path)

dataset = get_dataset('case_hold')

Found cached dataset lex_glue (/home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a)
100%|██████████| 3/3 [00:00<00:00, 375.51it/s]


In [10]:
dataset['train'].features.keys()



dict_keys(['context', 'endings', 'label'])

In [13]:
type(dataset['train']['context'])

data = {}
for split in ['train', 'test', 'validation']:
    keys = dataset[f'{split}'].features.keys()
    data[f'{split}'] = {}

    for key in keys:
        data[f'{split}'][f'{key}'] = dataset[f'{split}'][key]

list

In [ ]:
get_dataset('case_hold')

In [3]:
DATA_PATH = '/home/laal_intern003/LegalMaster/datast/case_hold.json'
data = load_dataset("json", data_files=DATA_PATH)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 336.97it/s]


Dataset json downloaded and prepared to /home/laal_intern003/.cache/huggingface/datasets/json/default-423ee07a41f4a944/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 280.20it/s]


In [4]:
type(data)

datasets.dataset_dict.DatasetDict

In [20]:
data_1 = load_dataset('lex_glue', 'case_hold')

Found cached dataset lex_glue (/home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a)
100%|██████████| 3/3 [00:00<00:00, 523.52it/s]


In [10]:
dataset = build_dataset(['case_hold'],'./dataset')

100%|██████████| 1/1 [00:03<00:00,  3.24s/it]

Datasets are succesfully loaded


In [3]:
if not os.path.exists(os.path.join(path, f'{case_hold}.pkl')):
    print('yes')
    with open(os.path.join(path, f'{task}.pkl'), 'wb') as f:
        pickle.dump(dataset, f)

NameError: name 'case_hold' is not defined

In [38]:
len(dataset)

1

In [1]:
path = './dataset'
os.path.exists(os.path.join(path, 'case_hold.pkl'))

False

In [5]:
with open('./dataset/case_hold.pkl', 'rb') as f:
    datast = pickle.load(f)

In [8]:
dataset[0]

DatasetDict({
    train: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3600
    })
    validation: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3900
    })
})

In [22]:
dataset[0]['train'].shuffle()['context'].map(tokenize

AttributeError: 'list' object has no attribute 'map'

In [19]:
# Load data
# data = load_dataset("json", data_files=./data)

tokenizer = LlamaTokenizer.from_pretrained(
    './llama',
    add_eos_token = True # add end-of-sentence token
)
tokenizer.pad_token_id = 0


def generate_prompt(data_point):
    return data_point["context"]

def tokenize(prompt,tokenizer, CUTOFF_LEN):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


def generate_and_tokenize_prompt(data_point, tokenizer, CUTOFF_LEN):
    prompt = generate_prompt(data_point)
    return tokenize(prompt, tokenizer, CUTOFF_LEN)



In [54]:
data = json.load(open('./data/quora_chat_data.json'))
data[:2] # dtype : list

[{'topic': 'What is the step by step guide to invest in share market in india?',
  'input': 'The conversation between human and AI assistant.\n[|Human|] I want to know the step by step guide to invest in share market in India.\n[|AI|] Sure, I can help with that. Firstly, you need to open a demat and trading account with a registered stockbroker.\n[|Human|] How do I find a registered stockbroker in India?\n[|AI|] You can visit the websites of National Stock Exchange (NSE) or Bombay Stock Exchange (BSE) to get a list of registered stockbrokers in India.\n[|Human|] What documents are required to open a demat and trading account?\n[|AI|] You will need to provide identity proof (PAN card), address proof, bank details and a passport size photograph to open a demat and trading account.\n[|Human|] How do I start trading once I have a demat and trading account?\n[|AI|] You can start trading by placing buy and sell orders for stocks through your stockbroker either online or offline.\n[|Human|] H

In [30]:
data[0]

{'topic': 'What is the step by step guide to invest in share market in india?',
 'input': 'The conversation between human and AI assistant.\n[|Human|] I want to know the step by step guide to invest in share market in India.\n[|AI|] Sure, I can help with that. Firstly, you need to open a demat and trading account with a registered stockbroker.\n[|Human|] How do I find a registered stockbroker in India?\n[|AI|] You can visit the websites of National Stock Exchange (NSE) or Bombay Stock Exchange (BSE) to get a list of registered stockbrokers in India.\n[|Human|] What documents are required to open a demat and trading account?\n[|AI|] You will need to provide identity proof (PAN card), address proof, bank details and a passport size photograph to open a demat and trading account.\n[|Human|] How do I start trading once I have a demat and trading account?\n[|AI|] You can start trading by placing buy and sell orders for stocks through your stockbroker either online or offline.\n[|Human|] How

In [41]:
json.dump(data, open('./data/tmp.json', 'w'))
data_r = load_dataset('json', data_files = './data/tmp.json')
data_r # dtype: datasetdict class

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 43.99it/s]


Dataset json downloaded and prepared to /home/laal_intern003/.cache/huggingface/datasets/json/default-3cd1d2c192d1f799/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 11.12it/s]


DatasetDict({
    train: Dataset({
        features: ['topic', 'input'],
        num_rows: 54456
    })
})

In [21]:

tokenizer = LlamaTokenizer.from_pretrained(
    './llama',
    add_eos_token = True # add end-of-sentence token
)
tokenizer.pad_token_id = 0
#data_r_mapped = data_r['train'].train_test_split(test_size = 2000, shuffle = True, seed = 42).shuffle().map(generate_and_tokenize_prompt)

In [4]:
import pickle


In [5]:
with open('./dataset/case_hold.pkl', 'rb') as f:
    case_hold = pickle.load(f)

EOFError: Ran out of input

In [14]:
case_hold = build_dataset(['case_hold'], './dataset')[0]

100%|██████████| 1/1 [00:02<00:00,  2.73s/it]

Datasets are succesfully loaded


In [24]:
new_case_hold = case_hold.shuffle().map(generate_and_tokenize_prompt)

In [27]:
case_hold['train'].shuffle()

Dataset({
    features: ['context', 'endings', 'label'],
    num_rows: 45000
})